## 150: Restricción de Acceso a Usuarios Verificados en FastAPI

Asegurar que solo los usuarios con cuenta verificada (is_active=True) puedan acceder a ciertos endpoints protegidos del sistema.

🧩 1. Middleware o Dependencia para Validar Verificación de Cuenta
Creamos una función que valida si el usuario está verificado antes de permitir acceso:

python
Copiar
Editar
from fastapi import Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from jose import jwt, JWTError

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Clave secreta y algoritmo del token
SECRET_KEY = "clave_supersecreta"
ALGORITHM = "HS256"

def obtener_usuario_actual(token: str = Depends(oauth2_scheme), db: Session = Depends(get_db)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        email: str = payload.get("sub")
        if email is None:
            raise HTTPException(status_code=401, detail="Credenciales inválidas")
    except JWTError:
        raise HTTPException(status_code=401, detail="Token inválido")

    user = db.query(User).filter_by(email=email).first()
    if user is None:
        raise HTTPException(status_code=404, detail="Usuario no encontrado")

    return user
🧩 2. Verificar que el usuario esté activo
Creamos una segunda función dependiente que valida el estado is_active del usuario:

python
Copiar
Editar
def verificar_usuario_activado(usuario: User = Depends(obtener_usuario_actual)):
    if not usuario.is_active:
        raise HTTPException(status_code=403, detail="🔒 Cuenta no verificada. Revisa tu correo.")
    return usuario
🧩 3. Aplicar restricción en rutas protegidas
Solo se permite acceso a usuarios verificados:

python
Copiar
Editar
@app.get("/dashboard")
def acceder_dashboard(usuario: User = Depends(verificar_usuario_activado)):
    return {"mensaje": f"👋 Bienvenido al panel, {usuario.email}"}